### 参数写入json

In [2]:
import json
kafka_params = {}
kafka_params['http'] = '10.131.142.24:19092'
kafka_params['topic'] = 'dc_gw_cmdb'
token_params = {}
token_params['login_url'] = "http://ywmh.sh.sgcc.com.cn/sso/api-token-auth/"
token_params['username'] = "mahayana"
token_params['password'] = "MAha_6997_yana"
cmdb_params = {}
cmdb_params['cmdb_url'] = 'http://ywmh.sh.sgcc.com.cn/cmdb/index/'
cmdb_params['query_url'] = 'http://ywmh.sh.sgcc.com.cn/cmdb/search/query_ci/'
cookies_params = {}
cookies_params['csrftoken'] = 'UDQdXIavaPPRRjd33ehW4Er0UWRjXizxKqb7KTwtIPRcYDkg7Fsxxt1UDolEIN3O'
cookies_params['X-CSRFToken'] = 'UDQdXIavaPPRRjd33ehW4Er0UWRjXizxKqb7KTwtIPRcYDkg7Fsxxt1UDolEIN3O'
params = {}
params['kafka'] = kafka_params
params['token'] = token_params
params['cmdb'] = cmdb_params
params['cookies'] = cookies_params
params

{'kafka': {'http': '10.131.142.24:19092', 'topic': 'dc_gw_cmdb'},
 'token': {'login_url': 'http://ywmh.sh.sgcc.com.cn/sso/api-token-auth/',
  'username': 'mahayana',
  'password': 'MAha_6997_yana'},
 'cmdb': {'cmdb_url': 'http://ywmh.sh.sgcc.com.cn/cmdb/index/',
  'query_url': 'http://ywmh.sh.sgcc.com.cn/cmdb/search/query_ci/'},
 'cookies': {'csrftoken': 'UDQdXIavaPPRRjd33ehW4Er0UWRjXizxKqb7KTwtIPRcYDkg7Fsxxt1UDolEIN3O',
  'X-CSRFToken': 'UDQdXIavaPPRRjd33ehW4Er0UWRjXizxKqb7KTwtIPRcYDkg7Fsxxt1UDolEIN3O'}}

In [3]:
with open('./util/config.json', 'w') as json_file:
    json_file.write(json.dumps(params)) # 相当于 json.dump(params, json_file)


In [5]:
from confluent_kafka import Producer
class Kafka(object):
    def __init__(self, config):
        # kafka produce
        self.p = Producer({"bootstrap.servers": config["kafka"]["http"],
                           "message.max.bytes": 20000000})

    def write(self, value, topic):
        # self.p.poll(0)
        self.p.produce(
            # config["kafka"]["topic.rca"],
            topic,
            json.dumps(value).encode("utf-8"),
            callback=self.delivery_report,
        )
        self.p.flush()
        
    @staticmethod
    def delivery_report(err, msg):
        if err is not None:
            print('消息发送失败:', err)
        else:
            print('消息发送成功:', msg.topic(), msg.partition())

In [ ]:
import requests, time
from util.log import LogHandler


class postdata:
    def __init__(self):
        with open('./util/config.json','r') as json_file:
            self.params = json.load(json_file)
        self.kafkaobj = Kafka(self.params)
        self.LogForest = LogHandler("query", "INFO").logger_basic()
        self.get_cmdb_token()     
    
    def get_cmdb_token(self):
        self.session = requests.Session()
        login_data = {'username': self.params['token']['username'], 'password': self.params['token']['password']}
        app_token = json.loads(self.session.post(self.params['token']['login_url'], login_data).content)['token']
        self.session.cookies.update({'app_token': app_token})
        self.session.cookies.update({'csrftoken': params['cookies']['csrftoken']})
        self.session.headers.update({'X-CSRFToken': params['cookies']['X-CSRFToken']})
        self.session.get(self.params['cmdb']['cmdb_url'])

    def query_ci(self, query):
        # 请求微服务参数  ，cicat_id: 272 代表微服务环境
        if query == 'microservice':
            query_data = {{"exact":{"value":"","flag":"fuzzy"},"cicat_id":[272],"order":{},"define":[{"input_type":"select","servicestatus":["在用"]},{"input_type":"text","ssfw|fqdn":""},{"input_type":"text","ssfw|name":""},{"input_type":"relaAttr","ssfw|YYXT_SHXT":""},{"input_type":"text","ssfw|Eservername":""},{"input_type":"text","ssfw|YYXT_SHXT|fqdn":""},{"input_type":"text","ssfw|YYXT_SHXT|name":""},{"input_type":"text","ssfw|YYXT_SHXT|systemenbrief":""},{"input_type":"select","ssfw|YYXT_SHXT|yyxtyxzt":""}]}}

        # 请求虚拟机参数 260,333代表虚拟机
        if query == 'linux':
            query_data = {"exact":{"value":"","flag":"fuzzy"},"cicat_id":[260,333],"order":{},"define":[{"input_type":"select","sbzt":["在运"]}]}

        # 请求应用服务（中间件）参数 包括kafka，es
        if query == 'middleware':
            query_data ={"exact":{"value":"","flag":"fuzzy"},"cicat_id":[268],"order":{},"define":[{"input_type":"select","servicestatus":["在用"]},{"input_type":"select","servicename|servicestatus":["在用"]},{"input_type":"text","servicename|serviceip":""},{"input_type":"text","servicename|service_domain":""},{"input_type":"text","servicename|serverport1":""},{"input_type":"text","run_on|fqdn":""},{"input_type":"text","run_on|name":""},{"input_type":"select","run_on|sbzt":""},{"input_type":"date","run_on|putdate":""},{"input_type":"textNum","run_on|cpuvirtualcores":""}]}

        # 请求数据库，包括mongodb，redis，postgres，mysql，oracle
        if query == 'database':
            query_data = {"exact":{"value":"","flag":"fuzzy"},"cicat_id":[273,274,320],"order":{},"define":[{"input_type":"select","servicestatus":["在用"]}]}
        result = json.loads(self.session.post(url=self.params['cmdb']['query_url'], json=query_data).content)
        return result['data']
    
    def main(self,querys):
        for query in querys:
            t = {'timestamp':time.time()}
            try:
                results = self.query_ci(query)
                for result in results:
                    result.update(t)
                    self.kafkaobj.write(result,self.params['kafka']['topic'])
                    self.LogForest.info(f"当前数据{result}")
            except Exception:
                self.LogForest.error(f"数据上传失败", exc_info=True)


In [6]:
kafka_params = {}
kafka_params['http'] = '192.168.50.110:19092'
kafka_params['topic'] = 'dc_gw_cmdb'
params = {'kafka':kafka_params}
kafkaobj = Kafka(params)
with open('/home/weijunfei/algorithm/交互/Kafka/history_170_40days.json') as f:
    data = json.load(f)

In [18]:
import time
from util.log import LogHandler
LogForest = LogHandler("query", "INFO").logger_basic()
t = {'timestamp':time.time()}
for i in data[:10]:
    i.update(t)
    kafkaobj.write(i, params['kafka']['topic'])
    LogForest.info(f"{i}")

2023-05-11 14:11:47 PM -<ipython-input-18-d5182f771430> -INFO[line:8]: -{'apmGroup': '*', 'dd-api-key': 'f186854e-8cad-3dfb-942a-dabf985c44e3', 'head': 170, 'monitorId': 170, 'name': 'request', 'scope': '{"type":"metric","metric":"secReqCnt","service":"kafka","serviceId":"ZjE4Njg1NGUtOGNhZC0zZGZiLTk0MmEtZGFiZjk4NWM0NGUzfGthZmth","resource":["*"]}', 'silence': 0, 'timestamp': 1683785507.5206678, 'value': 2}
2023-05-11 14:11:47 PM -<ipython-input-18-d5182f771430> -INFO[line:8]: -{'apmGroup': '*', 'dd-api-key': 'f186854e-8cad-3dfb-942a-dabf985c44e3', 'head': 170, 'monitorId': 170, 'name': 'request', 'scope': '{"type":"metric","metric":"secReqCnt","service":"kafka","serviceId":"ZjE4Njg1NGUtOGNhZC0zZGZiLTk0MmEtZGFiZjk4NWM0NGUzfGthZmth","resource":["*"]}', 'silence': 0, 'timestamp': 1683785507.5206678, 'value': 2}
2023-05-11 14:11:47 PM -<ipython-input-18-d5182f771430> -INFO[line:8]: -{'apmGroup': '*', 'dd-api-key': 'f186854e-8cad-3dfb-942a-dabf985c44e3', 'head': 170, 'monitorId': 170, 'name

消息发送成功: dc_gw_cmdb 0
消息发送成功: dc_gw_cmdb 0
消息发送成功: dc_gw_cmdb 0
消息发送成功: dc_gw_cmdb 2
消息发送成功: dc_gw_cmdb 2
消息发送成功: dc_gw_cmdb 2
消息发送成功: dc_gw_cmdb 2
消息发送成功: dc_gw_cmdb 0
消息发送成功: dc_gw_cmdb 0
消息发送成功: dc_gw_cmdb 0


In [6]:
a = [{'a':'1'},{'a':4},'a']
for i in a:
    try:
        if i.get('a'):
            print(i.get('a'))
    except:
        print(i)

1
4
a


In [8]:
import json
json.loads('')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)